Working around some `mistune` inconsistencies with doctype declaration and xml prolog

In [43]:
    import mistune,re
    renderer = mistune.Renderer()

The trouble is that the `mistune.BlockGrammar.block_html.pattern` misses both the doctype and prolog.  For my personal issue, links in the doctype are being autolinked and I don't want that. 

A bad example is shown below.  In this example:
* The prolog is escaped.
* The link in doctype is parsed.

In [45]:
    mistune.Markdown(renderer=renderer, escape=False, parse_block_html=True, parse_inline_html=True
    ).render("http://google.com\n\n" + graphviz.Source('graph {A}')._repr_svg_())

'<p><a href="http://google.com">http://google.com</a></p>\n<p>&lt;?xml version="1.0" encoding="UTF-8" standalone="no"?&gt;\n&lt;!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"\n "<a href="http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd&quot;&gt;">http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd&quot;&gt;</a>\n<!-- Generated by graphviz version 2.38.0 (20140413.2041)\n -->\n<!-- Title: %3 Pages: 1 --></p>\n<p>&lt;svg width="62pt" height="44pt"\n viewBox="0.00 0.00 62.00 44.00" xmlns="<a href="http://www.w3.org/2000/svg">http://www.w3.org/2000/svg</a>" xmlns:xlink="<a href="http://www.w3.org/1999/xlink&quot;&gt;">http://www.w3.org/1999/xlink&quot;&gt;</a></p>\n<p><g id="graph0" class="graph" transform="scale(1 1) rotate(0) translate(4 40)"></p>\n<p><title>%3</title></p>\n<p><polygon fill="white" stroke="none" points="-4,4 -4,-40 58,-40 58,4 -4,4"/>\n<!-- A --></p>\n<p><g id="node1" class="node"><title>A</title></p>\n<p><ellipse fill="none" stroke="black" cx="27" cy="-18" rx="27" ry="18"/>

The `replacement` adds conditions for the doctype and prolog to be added to both the `mistune.BlockGrammar.block_html.pattern` and `mistune.InlineGrammar.inline_html.pattern`.

In [47]:
    replacement = r'-->|<!DOCTYPE [^>]*?>|<\?[^>].*?\?>'

In [48]:
    class BlockGrammar(mistune.BlockGrammar):
        block_html = re.compile(mistune.BlockGrammar.block_html.pattern.replace('-->', replacement))
    class InlineGrammar(mistune.InlineGrammar):
        inline_html = re.compile(mistune.InlineGrammar.inline_html.pattern.replace('-->', replacement))

In [49]:
    mistune.Markdown(renderer=renderer, escape=False, parse_block_html=True, parse_inline_html=True,
        block=mistune.BlockLexer(BlockGrammar()), inline=mistune.InlineLexer(renderer, InlineGrammar())
    ).render(
        "http://google.com\n\n" + 
        graphviz.Source('graph {A}')._repr_svg_()
    )

'<p><a href="http://google.com">http://google.com</a></p>\n<p><?xml version="1.0" encoding="UTF-8" standalone="no"?>\n<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"\n "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">\n<!-- Generated by graphviz version 2.38.0 (20140413.2041)\n -->\n<!-- Title: %3 Pages: 1 --></p>\n<p>&lt;svg width="62pt" height="44pt"\n viewBox="0.00 0.00 62.00 44.00" xmlns="<a href="http://www.w3.org/2000/svg">http://www.w3.org/2000/svg</a>" xmlns:xlink="<a href="http://www.w3.org/1999/xlink&quot;&gt;">http://www.w3.org/1999/xlink&quot;&gt;</a></p>\n<p><g id="graph0" class="graph" transform="scale(1 1) rotate(0) translate(4 40)"></p>\n<p><title>%3</title></p>\n<p><polygon fill="white" stroke="none" points="-4,4 -4,-40 58,-40 58,4 -4,4"/>\n<!-- A --></p>\n<p><g id="node1" class="node"><title>A</title></p>\n<p><ellipse fill="none" stroke="black" cx="27" cy="-18" rx="27" ry="18"/></p>\n<p><text text-anchor="middle" x="27" y="-14.3" font-family="Times New Roman,serif" f